In [1]:
import os
import time
import load_data
import torch
import torch.nn.functional as F
from torch.autograd import Variable
import torch.optim as optim
import numpy as np
from models.RCNN_MTL2 import RCNN_MTL
import pickle

torch.manual_seed(999)
import random
random.seed(100)

In [2]:
TEXT, vocab_size, word_embeddings, train_iter, valid_iter, test_iter = load_data.load_dataset()



Length of Text Vocabulary: 382804
Vector size of Text Vocabulary:  torch.Size([382804, 100])
Label Length: 2
AGE BINS Label Length: 5


In [3]:
def clip_gradient(model, clip_value):
    params = list(filter(lambda p: p.grad is not None, model.parameters()))
    for p in params:
        p.grad.data.clamp_(-clip_value, clip_value)


def train_model(model, train_iter, epoch):

    total_epoch_loss = 0
    total_epoch_acc = 0
#     model.cuda()
    optim = torch.optim.Adam(filter(lambda p: p.requires_grad, model.parameters()))
    steps = 0
    model.train()
    for idx, batch in enumerate(train_iter):
        text = batch.text
        target = batch.label
        target2 = batch.age_bins
        target = torch.autograd.Variable(target).long()
        target2 = torch.autograd.Variable(target2).long()
        if torch.cuda.is_available():
            text = text.cuda()
            target = target.cuda()
            target2 = target2.cuda()
        if (text.size()[0] is not 32):# One of the batch returned by BucketIterator has length different than 32.
            continue
        optim.zero_grad()
        prediction = model(text)
        prediction_gender = prediction[0]
        prediction_age = prediction[1]
        loss1 = loss_fn(prediction_gender, target)
        loss2 = loss_fn(prediction_age, target2)
        loss = loss1 + loss2
        #*******To change below stuff *******
        num_corrects = (torch.max(prediction_gender, 1)[1].view(target.size()).data == target.data).float().sum()
        acc = 100.0 * num_corrects/len(batch)
        loss.backward()
        clip_gradient(model, 1e-1)
        optim.step()
        steps += 1

        if steps % 100 == 0:
            print (f'Epoch: {epoch+1}, Idx: {idx+1}, Training Loss: {loss.item():.4f}, Training Accuracy: {acc.item(): .2f}%')

        total_epoch_loss += loss.item()
        total_epoch_acc += acc.item()

    return total_epoch_loss/len(train_iter), total_epoch_acc/len(train_iter)

def eval_model(model, val_iter):
    total_epoch_loss = 0
    total_epoch_acc = 0
    model.eval()
    with torch.no_grad():
        for idx, batch in enumerate(val_iter):
            text = batch.text
            if (text.size()[0] is not 32):
                continue
            target = batch.label
            target2 = batch.age_bins
            target = torch.autograd.Variable(target).long()
            target2 = torch.autograd.Variable(target2).long()
            if torch.cuda.is_available():
                text = text.cuda()
                target = target.cuda()
                target2 = target2.cuda()
            prediction = model(text)
            prediction_gender = prediction[0]
            prediction_age = prediction[1]
            loss1 = loss_fn(prediction_gender, target)
            loss2 = loss_fn(prediction_age, target2)
            loss = loss1 + loss2
            num_corrects = (torch.max(prediction_gender, 1)[1].view(target.size()).data == target.data).sum()
            acc = 100.0 * num_corrects/len(batch)
            total_epoch_loss += loss.item()
            total_epoch_acc += acc.item()

    return total_epoch_loss/len(val_iter), total_epoch_acc/len(val_iter)






In [4]:

learning_rate = 2e-5
batch_size = 32
output_size = 2
output_size2 = 9
hidden_size = 256
embedding_length = 100

model = RCNN_MTL(batch_size, output_size, output_size2, hidden_size, vocab_size, embedding_length, word_embeddings)
loss_fn = F.cross_entropy



/Users/hiteshochani/anaconda3/envs/statnlp/lib/python3.6/site-packages/torch/nn/modules/rnn.py:38: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.8 and num_layers=1
  "num_layers={}".format(dropout, num_layers))


In [5]:

for epoch in range(20):
    train_loss, train_acc = train_model(model, train_iter, epoch)
    val_loss, val_acc = eval_model(model, valid_iter)

    pickle.dump(model, open("model_rcnn_idx"+str(epoch)+".pickle","wb"))
    print(f'Epoch: {epoch+1:02}, Train Loss: {train_loss:.3f}, Train Acc: {train_acc:.2f}%, Val. Loss: {val_loss:3f}, Val. Acc: {val_acc:.2f}%')



Epoch: 1, Idx: 100, Training Loss: 2.0854, Training Accuracy:  53.12%
Epoch: 1, Idx: 200, Training Loss: 1.7696, Training Accuracy:  68.75%
Epoch: 1, Idx: 300, Training Loss: 1.2828, Training Accuracy:  65.62%


/Users/hiteshochani/.local/lib/python3.6/site-packages/torchtext/data/field.py:322: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  return Variable(arr, volatile=not train)


Epoch: 01, Train Loss: 1.653, Train Acc: 62.68%, Val. Loss: 1.570326, Val. Acc: 65.21%
Epoch: 2, Idx: 100, Training Loss: 1.7296, Training Accuracy:  53.12%
Epoch: 2, Idx: 200, Training Loss: 1.2015, Training Accuracy:  68.75%
Epoch: 2, Idx: 300, Training Loss: 1.4796, Training Accuracy:  68.75%
Epoch: 02, Train Loss: 1.486, Train Acc: 69.02%, Val. Loss: 1.545994, Val. Acc: 66.17%
Epoch: 3, Idx: 100, Training Loss: 1.1808, Training Accuracy:  71.88%
Epoch: 3, Idx: 200, Training Loss: 1.6188, Training Accuracy:  68.75%
Epoch: 3, Idx: 300, Training Loss: 1.4098, Training Accuracy:  71.88%
Epoch: 03, Train Loss: 1.390, Train Acc: 71.69%, Val. Loss: 1.502102, Val. Acc: 69.39%
Epoch: 4, Idx: 100, Training Loss: 1.2573, Training Accuracy:  78.12%
Epoch: 4, Idx: 200, Training Loss: 1.4353, Training Accuracy:  78.12%
Epoch: 4, Idx: 300, Training Loss: 1.3358, Training Accuracy:  75.00%
Epoch: 04, Train Loss: 1.283, Train Acc: 74.89%, Val. Loss: 1.523204, Val. Acc: 68.64%
Epoch: 5, Idx: 100, Tr

In [6]:
test_loss, test_acc = eval_model(model, test_iter)
print(f'Test Loss: {test_loss:.3f}, Test Acc: {test_acc:.2f}%')

''' Let us now predict the sentiment on a single sentence just for the testing purpose. '''
test_sen1 = "This is one of the best creation of Nolan. I can say, it's his magnum opus. Loved the soundtrack and especially those creative dialogues."
test_sen2 = "Ohh, such a ridiculous movie. Not gonna recommend it to anyone. Complete waste of time and money."

test_sen1 = TEXT.preprocess(test_sen1)
test_sen1 = [[TEXT.vocab.stoi[x] for x in test_sen1]]

test_sen2 = TEXT.preprocess(test_sen2)
test_sen2 = [[TEXT.vocab.stoi[x] for x in test_sen2]]

test_sen = np.asarray(test_sen1)
test_sen = torch.LongTensor(test_sen)
test_tensor = Variable(test_sen, volatile=True)
test_tensor = test_tensor.cuda()
model.eval()
output = model(test_tensor, 1)
out = F.softmax(output, 1)
if (torch.argmax(out[0]) == 1):
    print ("Sentiment: Positive")
else:
    print ("Sentiment: Negative")

/Users/hiteshochani/.local/lib/python3.6/site-packages/torchtext/data/field.py:322: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  return Variable(arr, volatile=not train)


Test Loss: 3.234, Test Acc: 62.82%


/Users/hiteshochani/anaconda3/envs/statnlp/lib/python3.6/site-packages/ipykernel_launcher.py:16: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  app.launch_new_instance()


RuntimeError: Cannot initialize CUDA without ATen_cuda library. PyTorch splits its backend into two shared libraries: a CPU library and a CUDA library; this error has occurred because you are trying to use some CUDA functionality, but the CUDA library has not been loaded by the dynamic linker for some reason.  The CUDA library MUST be loaded, EVEN IF you don't directly use any symbols from the CUDA library! One common culprit is a lack of -Wl,--no-as-needed in your link arguments; many dynamic linkers will delete dynamic library dependencies if you don't depend on any of their symbols.  You can check if this has occurred by using ldd on your binary to see if there is a dependency on *_cuda.so library.